In [1]:
import numpy as np

In [2]:
def h_prima(H: int, mensaje: str) -> str:
    
    # Se divide el h0 en 4 partes de la siguiente forma
    mascara = 0xFFFFFFFF
    a0 = (H & (mascara << 96)) >> 96
    b0 = (H & (mascara << 64)) >> 64
    c0 = (H & (mascara << 32)) >> 32
    d0 = H & mascara
    
    # Se especifica los shifts por ronda
    s = []
    s[0:15] =  [ 7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22,  7, 12, 17, 22 ]
    s[16:31] = [ 5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20,  5,  9, 14, 20 ]
    s[32:47] = [ 4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23,  4, 11, 16, 23 ]
    s[48:63] = [ 6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21,  6, 10, 15, 21 ]
      
    k = []
    for i in range(64):
        k.append(int(np.floor(2**32*abs(np.sin(i + 1)))) & mascara)
    
    # Se inicializan las variables
    A = a0
    B = b0
    C = c0
    D = d0
    
    # Se divide el mensaje en chunks de 32 bits
    M = []
    for g in range(0,len(mensaje),4):
        M.append(int.from_bytes(mensaje[g:g+4], byteorder='little'))
    
    for j in range(64):
        F,g = 0,0
        if 0 <= j <= 15:
            F = (B & C) | ((~ B) & D)
            g = j
        elif 16 <= j <= 31:
            F = (D & B) | ((~ D) & C)
            g = (5*j + 1) % 16
        elif 32 <= j <= 47:
            F = B ^ C ^ D
            g = (3*j + 5) % 16
        elif 48 <= j <= 63:
            F = C ^ (B | (~ D))
            g = (7*j) % 16
        
        F = (F + A + k[j] + M[g]) & mascara
        A = D
        D = C
        C = B
        B = (B + (F << s[j] | F >> (32-s[j])) & mascara) & mascara
        
    a0 = (a0 + A) & mascara
    b0 = (b0 + B) & mascara
    c0 = (c0 + C) & mascara
    d0 = (d0 + D) & mascara

    return a0 + (b0 << 32) + (c0 << 64) + (d0 << 96)

In [3]:
def md5_to_hex(digest):
    raw = digest.to_bytes(16, byteorder='little')
    return '{:032x}'.format(int.from_bytes(raw, byteorder='big'))

In [4]:
def custom_md5(m: str, h0: int) -> str:
    # Argumentos:
    #  m: str - mensaje
    # h0: int - constante inicial H_0
    # Retorna:
    #  str - hash MD5 correcto del mensaje en formato hexadecimal
    
    # lo primero es dejar el mensaje divisible por 512
    
    # Ahora se agregarán un uno y ceros hasta que quede divisible por 512
        
    # Se agrega el largo original del mensaje
    message = bytearray(m, 'utf-8') #copy our input into a mutable buffer
    orig_len_in_bits = (8 * len(message)) & 0xffffffffffffffff
    message.append(0x80)
    while len(message)%64 != 56:
        message.append(0)
    message += orig_len_in_bits.to_bytes(8, byteorder='little')

        
    # Ahora se toman los estados para calcular el h_prima
    H = h0
    for i in range(0,len(message),64):
        H = h_prima(H, message[i: i + 64])
        
    return md5_to_hex(H)   

In [5]:
custom_md5("", 137269462086865085541390238039692956790)

'd41d8cd98f00b204e9800998ecf8427e'

## Referencias
1. https://rosettacode.org/wiki/MD5/Implementation#Python
2. https://en.wikipedia.org/wiki/MD5
3. https://tools.ietf.org/html/rfc1321
4. https://paginas.fe.up.pt/~ei10109/ca/md5.html
5. https://cse.unl.edu/~ssamal/crypto/genhash.php